# Exhaustive Search for N=2 Optimal Configuration

For N=2 with 180° rotation symmetry, we only need to search over:
- One angle θ (the other is θ+180°)
- Relative position of the two trees

This is a tractable 3D search (θ, dx, dy).

In [1]:
import numpy as np
from shapely.geometry import Polygon
from shapely.ops import unary_union
from scipy.optimize import minimize_scalar, minimize
import warnings
warnings.filterwarnings('ignore')

# Tree geometry
TX = np.array([0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125])
TY = np.array([0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5])

def get_tree_polygon(cx, cy, angle_deg):
    angle_rad = np.radians(angle_deg)
    cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)
    x = TX * cos_a - TY * sin_a + cx
    y = TX * sin_a + TY * cos_a + cy
    return Polygon(zip(x, y))

def get_bounding_box_side(trees):
    all_poly = unary_union(trees)
    bounds = all_poly.bounds
    return max(bounds[2] - bounds[0], bounds[3] - bounds[1])

def check_overlap(p1, p2):
    if p1.intersects(p2):
        intersection = p1.intersection(p2)
        return intersection.area > 1e-10
    return False

print("Functions defined.")

Functions defined.


In [2]:
# Current N=2 solution
current_angles = [203.629, 23.629]  # 180° apart
current_positions = [(0.154, -0.039), (-0.154, -0.561)]

trees = [get_tree_polygon(x, y, a) for (x, y), a in zip(current_positions, current_angles)]
current_side = get_bounding_box_side(trees)
current_score = current_side**2 / 2

print(f"Current N=2 solution:")
print(f"  Angles: {current_angles}")
print(f"  Positions: {current_positions}")
print(f"  Side: {current_side:.6f}")
print(f"  Score: {current_score:.6f}")

Current N=2 solution:
  Angles: [203.629, 23.629]
  Positions: [(0.154, -0.039), (-0.154, -0.561)]
  Side: 0.949312
  Score: 0.450597


In [3]:
def optimize_positions_for_angle(theta):
    """
    For a given angle theta, find the optimal positions for two trees
    at angles theta and theta+180.
    
    We place tree 1 at origin and optimize position of tree 2.
    """
    def objective(params):
        dx, dy = params
        
        # Tree 1 at origin with angle theta
        tree1 = get_tree_polygon(0, 0, theta)
        # Tree 2 at (dx, dy) with angle theta+180
        tree2 = get_tree_polygon(dx, dy, theta + 180)
        
        # Check overlap
        if check_overlap(tree1, tree2):
            return 1000.0  # Penalty
        
        # Return bounding box side
        return get_bounding_box_side([tree1, tree2])
    
    # Try multiple starting points
    best_result = None
    best_side = float('inf')
    
    # Starting points based on tree geometry
    starts = [
        (0.5, 0.5), (-0.5, 0.5), (0.5, -0.5), (-0.5, -0.5),
        (0.3, 0.6), (-0.3, 0.6), (0.3, -0.6), (-0.3, -0.6),
        (0.0, 0.8), (0.0, -0.8), (0.8, 0.0), (-0.8, 0.0),
        (0.2, 0.4), (-0.2, 0.4), (0.2, -0.4), (-0.2, -0.4),
    ]
    
    for start in starts:
        result = minimize(objective, start, method='Nelder-Mead', 
                         options={'maxiter': 500, 'xatol': 1e-8, 'fatol': 1e-8})
        if result.fun < best_side:
            best_side = result.fun
            best_result = result
    
    return best_side, best_result.x if best_result else None

print("Optimizer defined.")

Optimizer defined.


In [ ]:
# Exhaustive search over angles
print("Searching over all angles from 0° to 180° in 0.5° increments...")
print("(Due to 180° symmetry, we only need to search 0-180)")

best_angle = None
best_side = float('inf')
best_positions = None
results = []

for theta in np.arange(0, 180, 0.5):
    side, pos = optimize_positions_for_angle(theta)
    results.append((theta, side, pos))
    
    if side < best_side:
        best_side = side
        best_angle = theta
        best_positions = pos
        print(f"  New best at θ={theta:.1f}°: side={side:.6f}")

print(f"\nBest angle: {best_angle}°")
print(f"Best side: {best_side:.6f}")
print(f"Best score: {best_side**2/2:.6f}")
print(f"Best positions: tree2 at ({best_positions[0]:.6f}, {best_positions[1]:.6f})")

In [ ]:
# Fine-tune around the best angle
print(f"\nFine-tuning around θ={best_angle}°...")

for theta in np.arange(best_angle - 1, best_angle + 1, 0.01):
    side, pos = optimize_positions_for_angle(theta)
    
    if side < best_side - 1e-8:
        best_side = side
        best_angle = theta
        best_positions = pos
        print(f"  Improved at θ={theta:.2f}°: side={side:.8f}")

print(f"\nFinal best angle: {best_angle:.4f}°")
print(f"Final best side: {best_side:.8f}")
print(f"Final best score: {best_side**2/2:.8f}")

In [ ]:
# Compare with current solution
print("\n" + "="*60)
print("COMPARISON WITH CURRENT SOLUTION")
print("="*60)
print(f"Current N=2 score: {current_score:.8f}")
print(f"Best found score:  {best_side**2/2:.8f}")
print(f"Improvement:       {current_score - best_side**2/2:.8f}")

if best_side**2/2 < current_score - 1e-8:
    print("\n*** IMPROVEMENT FOUND! ***")
    print(f"New configuration:")
    print(f"  Tree 1: (0, 0) at angle {best_angle:.4f}°")
    print(f"  Tree 2: ({best_positions[0]:.8f}, {best_positions[1]:.8f}) at angle {best_angle+180:.4f}°")
else:
    print("\nNo improvement found. Current solution is optimal or near-optimal.")

In [ ]:
# Visualize the best configuration
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Current solution
ax = axes[0]
for (x, y), a in zip(current_positions, current_angles):
    tree = get_tree_polygon(x, y, a)
    xs, ys = tree.exterior.xy
    ax.fill(xs, ys, alpha=0.5)
    ax.plot(xs, ys, 'b-', linewidth=0.5)
ax.set_aspect('equal')
ax.set_title(f'Current: side={current_side:.6f}, score={current_score:.6f}')
ax.grid(True, alpha=0.3)

# Best found solution
ax = axes[1]
tree1 = get_tree_polygon(0, 0, best_angle)
tree2 = get_tree_polygon(best_positions[0], best_positions[1], best_angle + 180)
for tree in [tree1, tree2]:
    xs, ys = tree.exterior.xy
    ax.fill(xs, ys, alpha=0.5)
    ax.plot(xs, ys, 'b-', linewidth=0.5)
ax.set_aspect('equal')
ax.set_title(f'Best found: side={best_side:.6f}, score={best_side**2/2:.6f}')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('/home/code/experiments/010_n2_exhaustive/n2_comparison.png', dpi=100)
plt.show()
print("Saved visualization.")

In [ ]:
# Save metrics
import json

metrics = {
    'cv_score': 70.659437,  # Overall score unchanged if no improvement
    'n2_current_score': current_score,
    'n2_best_found_score': best_side**2/2,
    'n2_improvement': current_score - best_side**2/2,
    'best_angle': best_angle,
    'best_positions': list(best_positions) if best_positions is not None else None,
    'notes': 'Exhaustive search for N=2 optimal configuration'
}

with open('/home/code/experiments/010_n2_exhaustive/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print(f"Metrics saved.")
print(f"N=2 improvement: {metrics['n2_improvement']:.8f}")